因为业务需要，每天都需要按当天的日期创建很多新文件夹。把这种重复又繁重的操作交给Python来做，一直是我的目标。先说下要求：
1. 默认在桌面新建文件夹。
2. 文件夹命名方式，“月.日-1”，比如7.8-1，7.8-2。7是代表当天的月份，不能显示07。8是代表当天的日期，不能显示08。-1,-2....是文件夹的序号。
3. 序号支持手动输入，比方可以自己输入10就创建10个文件夹。要有记忆功能，这次输入10以后，下次运行程序就默认是10。


再说下思路及步骤：
1. 使用Tkinter创建操作界面，需要有文件及路径文本框和选择按钮；文件夹数量输入框及执行新建按钮；显示操作执行过程的列表框。
2. 获取当天的月份和日期。
3. 按需求的数量创建新文件夹。
4. 将最近输入的文件夹数量保存到log.txt文件，下次运行程序直接从log.txt取数，实现记忆功能。

使用Tkinter创建界面，可参考实例63（https://zhuanlan.zhihu.com/p/151754669） 。下面来看看如何获取当天的月份和日期，并整理成想要的格式。定义了一个函数`get_current_date`，然后将日期转换成本地时间，再分别获取月份和天数，并转为整数，这样“07”才能变成“7”。最后将月和天用“.”连起来，就得到了最终结果。比如今天，结果就显示为"7.9"。

In [5]:
#获取当天的月份和日期，整理成想要的格式
from time import time, localtime ,strftime
def get_current_date():
    time_stamp = time()  # 当前时间的时间戳
    local_time = localtime(time_stamp)  #转为本地时间
    str_time_month = int(strftime('%m', local_time))#获取月份并转为整数
    str_time_day = int(strftime('%d', local_time))#获取天数并转为整数
    return str(str_time_month)+"."+str(str_time_day)#整理格式
get_current_date()

'7.9'

然后写创建新文件夹的函数`create_folders`，其中传入两个参数，即以上处理好的日期`date`和文件夹数量`qty`。用`for`循环根据数量来创建。先将文件夹的名称字符串按需要连接好，存入变量`folder`。然后用`if`语句判断此目录是否存在，若不存在，才使用`os.makedirs`创建，若存在，则输出“目录已存在”的提示。调用函数实验了一下，没问题。

In [41]:
#创建新文件夹
import os
def create_folders(date,qty):
    for i in range(1,qty+1):
        folder = 'D:\\' + date + '-' + str(i)
        # 判断是否已经存在该目录
        if not os.path.exists(folder):
            # 目录不存在，进行创建操作
            os.makedirs(folder) #使用os.makedirs()方法创建多层目录
            print("目录新建成功：" + folder) 
        else:
            print("目录已存在！")
                
date = get_current_date()
qty = 3
create_folders(date,qty)

目录新建成功：D:\7.9-1
目录新建成功：D:\7.9-2
目录新建成功：D:\7.9-3


下面定义一个函数`save_recent_folder_qty`，用于将最近使用过的“文件夹数量”保存到日志文件`log.txt`。先设定好文件的路径及文件名，存入变量`log_file`，然后使用写入模式`w`打开文件，将数量`qty`写入。写好后，调用一下，就可以在指定的路径下看到这个文件了，打开可见里面的内容为“10”。

In [42]:
def save_recent_folder_qty(qty):
    '''保存最近的文件夹数量'''
    log_file = os.getcwd() + "\\log.txt"
    with open(log_file, "w") as f:
        recent_qty = str(qty)
        f.write(recent_qty) 
save_recent_folder_qty(10)

然后还需要从上面保存的日志文件中提取最近用过的“文件夹数量”，还是定义一个函数`get_folder_qty`，指定路径存入`log_file`变量。首次运行程序的时候，这个日志文件还没有的。所以为避免报错，先用`if`语句判断是否存在。若已存在，则直接读取到`qty`变量中，并返回；如不存在，则返回“0”。写好后，也可以调用一下，看看是否有问题。

In [21]:
def get_folder_qty():
    log_file = os.getcwd() + "\\log.txt"
    if os.path.exists(log_file):
        with open(log_file, "r") as f:
            qty = f.readline()
            return int(qty)
    else:
        return 0
a = get_folder_qty()
print(a)

0


以上准备工作做好后，就可以整合到类`MainGUI`中去了。这个整合，基本上跟实例63（https://zhuanlan.zhihu.com/p/151754669） 差不多，下面挑有差异的部分进行说明。首先，增加了两个标签`self.label_1`和`self.label_2`，用于提示文本框中显示的内容，分别是“目标目录”和“文件件数量”。还有，就是给文本框增加了默认显示的内容。其中`StringVar`用于跟踪字符串变量，方便将其值显示在界面上。将要要显示的内容经`StringVar`处理，传给变量`addr`，然后赋给`Entry`中的`textvariable`参数，就可以在文本框显示了。同样，日志文件`log.txt`中读取最近使用过的文件夹数量也是这样显示在文本框。
<br/>
<br/>最后，将函数跟界面的按钮，文本框，列表框连接起来，就可以开始运行了。

In [44]:
import os
from datetime import datetime
from time import time, localtime ,strftime
from tkinter import Tk,Entry,Button,Listbox,X,Y,END,Scrollbar,RIGHT,BOTTOM,HORIZONTAL,StringVar,Label
from tkinter.filedialog import askdirectory

class MainGUI():
    def __init__(self):
        myWindow = Tk()
        myWindow.title("批量新建文件夹")
        #设置窗口大小
        myWindow.geometry('590x400')
        #增加标签
        self.label_1 = Label(myWindow, text=' 目标目录:')
        self.label_1.place(x=10, y=10,width=70, height=30)
        self.label_2 = Label(myWindow, text='文件夹数量:')
        self.label_2.place(x=10, y=50,width=70, height=30)

        #增加文本框
        addr = StringVar(value='C:\\Users\\xxxx\\Desktop') #文本框默认显示的内容
        self.input_entry = Entry(myWindow, highlightcolor='red', highlightthickness=1, textvariable=addr)
        self.input_entry.place(x=80, y=10,width=410, height=30)
        self.btn_in = Button(myWindow, text='选择目录',command = self.select_dir1, width=10, height=1) 
        self.btn_in.place(x=500,y=10)
        
        folder_quantity = str(self.get_folder_qty()) #从日志文件`log.txt`中读取最近使用过的文件夹数量
        def_qty = StringVar(value = folder_quantity)
        self.folderQty_entry = Entry(myWindow, highlightcolor='blue', highlightthickness=1, textvariable=def_qty)
        self.folderQty_entry.place(x=80, y=50,width=410, height=30)
        self.btn_exe = Button(myWindow, text='执行新建',command = self.create_folders, width=10, height=1)
        self.btn_exe.place(x=500,y=50)
        
        #增加列表框
        self.result_show = Listbox(myWindow,bg='Azure') 
        self.result_show.place(x=10,y=100, width=570, height=290)
        self.sbY = Scrollbar(self.result_show,command=self.result_show.yview)#在列表框中增加Y轴滚动条
        self.sbY.pack(side=RIGHT,fill=Y)
        self.result_show.config(yscrollcommand = self.sbY.set)
        self.sbX = Scrollbar(self.result_show,command=self.result_show.xview,orient = HORIZONTAL)#在列表框中增加X轴滚动条
        self.sbX.pack(side=BOTTOM,fill=X)
        self.result_show.config(xscrollcommand = self.sbX.set)
        
        myWindow.mainloop()
        
    def select_dir1(self):
        self.input_entry.delete(0, END)
        self.input_entry.insert(0, askdirectory(initialdir= "D:\\"))
        
    def create_folders(self):
        date = self.get_current_date() #获取日期
        qty = int(self.folderQty_entry.get()) #获取文本框中文件夹数量，并转为整数
        for i in range(1,qty+1):
            folder = self.input_entry.get() + "\\" + date + '-' + str(i)
            # 判断是否已经存在该目录
            if not os.path.exists(folder):
                # 目录不存在，进行创建操作
                os.makedirs(folder) #使用os.makedirs()方法创建目录
                f = f"目录新建成功：{folder}" # 创建一个显示项
                self.result_show.insert("end", f) #将结果添加到列表框中
            else:
                f = f"目录已存在：{folder}" # 创建一个显示项
                self.result_show.insert("end", f) #将结果添加到列表框中
        f = "-"*100 #创建分割线
        self.result_show.insert("end", f) # 将分割线添加到列表框
        f = f"程序运行完成，请关闭窗口退出."# 创建一个显示项
        self.result_show.insert("end", f) # 将结果添加到列表框
        f = " "*100
        self.result_show.insert("end", f) # 将以上空格添加到列表框
        self.save_recent_folder_qty() #保存最新的文件夹数量
        
    def get_current_date(self):
        time_stamp = time()  
        local_time = localtime(time_stamp)  
        str_time_month = int(strftime('%m', local_time))
        str_time_day = int(strftime('%d', local_time))
        return str(str_time_month)+"."+str(str_time_day)
    
    def get_folder_qty(self):
        '''从log.txt文件中获取最近的文件夹数量，若没有则返回0'''
        log_file = os.getcwd() + "\\log.txt"
        if os.path.exists(log_file):
            with open(log_file, "r") as f:
                qty = f.readline()
                return int(qty)
        else:
            return 0
        
    def save_recent_folder_qty(self):
        '''保存最近的文件夹数量'''
        log_file = os.getcwd() + "\\log.txt"
        with open(log_file, "w") as f:
            recent_qty = str(self.folderQty_entry.get())
            f.write(recent_qty) 
            
if __name__ == "__main__":
    MainGUI()

首次运行，弹出窗口如下。可见“目标目录”是我们设定好的默认显示值。因为第一次运行，还没有日志文件`log.txt`，所以“文件夹数量”是我们设定好的默认值“0”。

![](images\first_run.png)

选择好“目标目录”，并设定好“文件夹数量”后，点击“执行新建”，需要的文件夹秒秒钟建好。日志文件也将输入的“文件夹数量”自动保存下来了。
![](images\\result.PNG)